# Distressed order by tokenizing

- https://adventofcode.com/2022/day/13

Because you don't need to have the whole packets to decide if they are out of order, I decided to tokenize the inputs; process them character by character and compare the tokens of the two packets together. The only complication you have to handle is when one of the tokens is an opening bracket while the other is a number; you have to re-process the number plus a closing bracket each time.


In [1]:
from collections import deque
from dataclasses import dataclass
from typing import Literal, Self


class PacketTokenizer:
    def __init__(self, packet: str):
        self._it = iter(packet)
        self._pushback: deque[Literal["]"] | int] = deque()

    def __iadd__(self, number: int) -> Self:
        self._pushback.extend(("]", number))
        return self

    def __iter__(self) -> Self:
        return self

    def __next__(self) -> Literal["[", "]"] | int:
        if self._pushback:
            return self._pushback.pop()
        while True:
            match next(self._it):
                case ",":
                    continue
                case "[" | "]" as tok:
                    return tok
                case digit:
                    number = int(digit)
                    while True:
                        match next(self._it):
                            case "]":
                                self._pushback.append("]")
                                break
                            case ",":
                                break
                            case digit:
                                number = number * 10 + int(digit)
                    return number


@dataclass
class Packet:
    text: str

    def __lt__(self, other: Self) -> bool:
        if not isinstance(other, __class__):
            return NotImplemented
        p1, p2 = PacketTokenizer(self.text), PacketTokenizer(other.text)
        for t1, t2 in zip(p1, p2):
            match (t1, t2):
                case ("[", "[") | ("]", "]"):
                    pass
                case ("]", _):  # left is shorter
                    return True
                case (_, "]"):  # right is shorter
                    return False
                case (int() as number, "["):  # wrap left number in a list
                    p1 += number
                case ("[", int() as number):  # wrap right number in a list
                    p2 += number
                case (int() as n1, int() as n2) if n1 != n2:  # numbers are not equal
                    return n1 < n2
                case _:  # numbers are equal
                    pass
        return False  # equal, so not lower


def paired_packets(data: str) -> list[tuple[Packet, Packet]]:
    return [
        tuple(Packet(packet) for packet in pair.splitlines())
        for pair in data.split("\n\n")
    ]


example = paired_packets(
    """\
[1,1,3,1,1]
[1,1,5,1,1]

[[1],[2,3,4]]
[[1],4]

[9]
[[8,7,6]]

[[4,4],4,4]
[[4,4],4,4,4]

[7,7,7,7]
[7,7,7]

[]
[3]

[[[]]]
[[]]

[1,[2,[3,[4,[5,6,7]]]],8,9]
[1,[2,[3,[4,[5,6,0]]]],8,9]
"""
)


assert sum(i for i, (p1, p2) in enumerate(example, 1) if p1 < p2) == 13

In [2]:
import aocd

pairs = paired_packets(aocd.get_data(day=13, year=2022))
print("Part 1:", sum(i for i, (p1, p2) in enumerate(pairs, 1) if p1 < p2))

Part 1: 5605


## Part 2, more ordering

We have everything we need for part 2, because in Python we can sort any list of objects that implement `__lt__` to define an ordering. All we need to do is put the packets into a single list, add the divider packets and sort. Because I used dataclasses as the base (which support equality testing out of the box) finding the decoder packet indices is as simple as using `list.index()` (just remember to add 1 to turn the 0-based index into a 1-based index).


In [3]:
from itertools import chain


def decoder_key(pairs: list[tuple[Packet, Packet]]) -> int:
    divider_packets = Packet("[[2]]"), Packet("[[6]]")
    ordered = sorted(chain.from_iterable((divider_packets, *pairs)))
    return (ordered.index(divider_packets[0]) + 1) * (
        ordered.index(divider_packets[1]) + 1
    )


assert decoder_key(example) == 140

In [4]:
print("Part 2:", decoder_key(pairs))

Part 2: 24969
